In [ ]:
%pip install path_vis numpy

# Piecewise Constant Curvature (PCC) with standard state parameterization

kappa (𝜅), phi (𝜙), l (ℓ)

In [ ]:
import numpy as np

def rotz(phi):
    c, s = np.cos(phi), np.sin(phi)
    R = np.eye(4)
    R[:2, :2] = [[c, -s], [s, c]]
    return R

def roty(theta):
    c, s = np.cos(theta), np.sin(theta)
    R = np.eye(4)
    R[0, 0], R[0, 2] = c, s
    R[2, 0], R[2, 2] = -s, c
    return R

def segment_matrix(k, phi, s):
    if abs(k) < 1e-9:
        seg = np.eye(4)
        seg[2, 3] = s
    else:
        theta, r = k * s, 1 / k
        seg = roty(theta)
        seg[0, 3] = r * (1 - np.cos(theta))
        seg[2, 3] = r * np.sin(theta)
        seg = rotz(phi) @ seg @ rotz(-phi)
    return seg


def pcc_standard(kappa, l, phi, n=10):
    from path_vis import PathVis
    
    curr_frame = np.eye(4)
    curve = [ curr_frame.copy() ]

    for ki, li, pi in zip(kappa, l, phi):
        for j in range(1, n + 1):
            seg = curr_frame @ segment_matrix(ki, pi, j * li / n)
            curve.append(seg)
        curr_frame @= segment_matrix(ki, pi, li)

    flat_curve = np.concatenate([c.ravel() for c in curve]).tolist()
    return PathVis(flat_curve)

Random configurations

In [ ]:
rng = np.random.default_rng()
kappa = rng.uniform(0,20,3)
phi = rng.uniform(0, 2*np.pi, 3)
l = rng.uniform(0.1,0.3,3)

pcc_standard(kappa, l, phi)

Notice that even with varying values of phi (𝜙) each segment remains straight

In [ ]:
kappa = np.zeros(3)
phi = rng.uniform(0, 2*np.pi, 3)
l = np.array([0.2] * 3)

print(phi)
pcc_standard(kappa, l, phi)

# q-parameterization

Dx (Δx), Dy (Δy), dL (δL)

In [ ]:
import numpy as np

def segment_matrix_q(Dx, Dy, dL, L0=0.01, d=1.0):
    seg = np.eye(4)
    Delta = np.sqrt(Dx**2 + Dy**2)
    L = L0 + dL

    if Delta < 1e-9:
        # straight configuration (well-defined limit)
        seg[2, 3] = L
        return seg

    c = np.cos(Delta / d)
    s = np.sin(Delta / d)

    # Rotation
    R = np.array([
        [1 + (Dx**2/Delta**2)*(c-1), (Dx*Dy/Delta**2)*(c-1), -Dx/Delta*s],
        [(Dx*Dy/Delta**2)*(c-1), 1 + (Dy**2/Delta**2)*(c-1), -Dy/Delta*s],
        [Dx/Delta*s, Dy/Delta*s, c]
    ])

    seg[:3, :3] = R

    # Translation
    seg[0, 3] = d * L / (Delta**2) * Dx * (1 - c)
    seg[1, 3] = d * L / (Delta**2) * Dy * (1 - c)
    seg[2, 3] = d * L / (Delta**2) * Delta * s
    return seg


def pcc_q(Dx, Dy, dL, n=10):
    from path_vis import PathVis
    curr_frame = np.eye(4)
    curve = [curr_frame.copy()]

    for dx, dy, dl in zip(Dx, Dy, dL):
        dx_s = dx / n
        dy_s = dy / n
        dl_s = dl / n

        for _ in range(n):
            seg = curr_frame @ segment_matrix_q(dx_s, dy_s, dl_s)
            curve.append(seg)
            curr_frame = seg

    flat_curve = np.concatenate([c.ravel() for c in curve]).tolist()
    return PathVis(flat_curve)


In [ ]:
rng = np.random.default_rng()
Dx = rng.uniform(-2, 2, 3)
Dy = rng.uniform(-2, 2, 3)
dL = rng.uniform(0.1, 0.3, 3)

pcc_q(Dx, Dy, dL)

Notice that unless both Dx and Dy are zero there is still bending

In [ ]:
Dx = rng.uniform(-2, 2, 3)
Dy = np.zeros(3)
dL = np.array([0.2] * 3)

pcc_q(Dx, Dy, dL)

In [ ]:
Dx = np.zeros(3)
Dy = np.zeros(3)
dL = np.array([0.2] * 3)

pcc_q(Dx, Dy, dL)